# HUK-Coding Challenge


__Contents__

1. Explorative Datenanalyse
2. Modelbuilding
3. Evaluation

In [1]:
import pandas as pd
import os
from huk_challenge.config import ProjectPaths

In [ ]:

train = pd.read_csv(os.path.join(ProjectPaths.data, "training.csv"))